In [ ]:
import sys
import os
from types import SimpleNamespace

import numpy as np
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import concatenate as coord_concat
from astropy.io import ascii
import scipy.ndimage
import tqdm
%pylab inline

import constants
import lyafxcorr_kg as xcorr

In [ ]:
cosmo = constants.COSMOLOGY

PiBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins23_pi_0-30hMpc.txt')
SigBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins10_sigma_0-30hMpc.txt')

PiBins0 = ascii.read(PiBin_fil)
SigBins0 = ascii.read(SigBin_fil)

PiEdges = PiBins0['pi_edges'].data
SigEdges = SigBins0['sigma_edges'].data

# Convert bin boundaries from Mpc/h to Mpc
PiEdges  = PiEdges/(len(PiEdges)*[cosmo.h])
SigEdges = SigEdges/(len(SigEdges)*[cosmo.h])

In [ ]:
BIRTH_DIR = '/global/homes/b/bzh/clamato-xcorr/data/birth'

os.chdir(BIRTH_DIR)
import coord_conversion

# BIRTH constrained simulation x-corr (within footprint)

In [ ]:
CLAMATO_FOOTPRINT_RA = (149.89 * u.deg, 150.4 * u.deg)
CLAMATO_FOOTPRINT_DEC = (2.09 * u.deg, 2.5 * u.deg)
FOOTPRINT_PATH = 'clamato_footprint_mask.npy'

if os.path.exists(FOOTPRINT_PATH):
    clamato_footprint_mask = np.load(FOOTPRINT_PATH)
else:
    clamato_footprint_mask = np.zeros_like(coord_conversion.sm_delta)

    indices = []
    for x in range(0, coord_conversion.sm_delta.shape[0], 3):
        for y in range(0, coord_conversion.sm_delta.shape[1], 3):
            for z in range(0, coord_conversion.sm_delta.shape[2], 3):
                indices.append(((x + 1), (y + 1), (z + 1)))

    # indices = [(x, y, z) for x in range(coord_conversion.sm_delta.shape[0]) for y in range(coord_conversion.sm_delta.shape[1]) for z in range(coord_conversion.sm_delta.shape[2])]

    for ind in tqdm.tqdm(indices):
        x, y, z = ind
        coord = coord_conversion.original_pos(*coord_conversion.index2coord(x, y, z))
        if (not coord.ra) or (not coord.dec):
            print(f'Coordinate for ({x}, {y}, {z}), does not have RA or DEC; skipping.')
            continue
        if (2.05 <= coord.distance <= 2.55
            and CLAMATO_FOOTPRINT_RA[0] <= coord.ra <= CLAMATO_FOOTPRINT_RA[1] 
            and CLAMATO_FOOTPRINT_DEC[0] <= coord.dec <= CLAMATO_FOOTPRINT_DEC[1]):
            clamato_footprint_mask[x-1:x+2, y-1:y+2, z-1:z+2] = 1
    np.save(FOOTPRINT_PATH, clamato_footprint_mask)
                
print(f'Mask covers {np.sum(clamato_footprint_mask) / clamato_footprint_mask.size * 100}% of volume.')

In [ ]:
plt.figure(figsize=(10,10))
z = 126
plt.imshow(clamato_footprint_mask[z, :, :])

In [ ]:
redshift_mask = np.where(clamato_footprint_mask[:, 128, 128])[0]
print(len(redshift_mask))

In [ ]:
stamp_size = (np.argwhere(clamato_footprint_mask[126, :, 128]).size, np.argwhere(clamato_footprint_mask[126, 128, :]).size)
print(stamp_size)

In [ ]:
print(np.argwhere(clamato_footprint_mask[126, :, 128]))
print(np.argwhere(clamato_footprint_mask[126, 128, :]))

In [ ]:
clamato_footprint_bounds = [(np.argwhere(clamato_footprint_mask[:, 128, 128]).min(), np.argwhere(clamato_footprint_mask[:, 128, 128]).max()),
                            (np.argwhere(clamato_footprint_mask[126, :, 128]).min(), np.argwhere(clamato_footprint_mask[126, :, 128]).max()), 
                            (np.argwhere(clamato_footprint_mask[126, 128, :]).min(), np.argwhere(clamato_footprint_mask[126, 128, :]).max())]
print(clamato_footprint_bounds)

In [ ]:
plt.imshow(clamato_footprint_mask[z, :, :])
plt.axvline(123, color='red')
plt.axhline(120, color='red')

In [ ]:
clamato_footprint_sm_delta = np.mean(coord_conversion.sm_delta[clamato_footprint_mask.astype(np.bool)])
# volume_footprint_sm_delta = np.mean(coord_conversion.sm_delta[~clamato_footprint_mask.astype(np.bool)])
print(clamato_footprint_sm_delta)
# print(volume_footprint_sm_delta)

In [ ]:
bootstrap_iter = 100000
null_means = []
max_stamp = None
max_mean = -np.inf
np.random.seed(42)
for _ in tqdm.tqdm(range(bootstrap_iter)):
    stamp_corner = (np.random.randint(0, coord_conversion.sm_delta.shape[1] - stamp_size[0]), np.random.randint(0, coord_conversion.sm_delta.shape[2] - stamp_size[1]))
    null_means.append(np.mean(coord_conversion.sm_delta[redshift_mask, stamp_corner[0]:stamp_corner[0] + stamp_size[0], stamp_corner[1]:stamp_corner[1] + stamp_size[1]]))
    if null_means[-1] > max_mean:
        max_stamp = stamp_corner
        max_mean = null_means[-1]

In [ ]:
plt.hist(null_means, bins=20, color='black');
plt.axvline(clamato_footprint_sm_delta, color='red')
print((clamato_footprint_sm_delta - np.mean(null_means)) / np.std(null_means))

In [ ]:
print(max_stamp, max_mean)

In [ ]:
def cut_out_stamp(stamp):
    max_stamp_sm_delta = np.copy(coord_conversion.sm_delta)
    mask = np.zeros_like(max_stamp_sm_delta).astype(bool)
    mask[redshift_mask, stamp[0]:stamp[0] + stamp_size[0], stamp[1]:stamp[1] + stamp_size[1]] = True
    return max_stamp_sm_delta[mask].reshape(len(redshift_mask), *stamp_size)

# print(max_stamp_sm_delta.shape)
fig, axes = plt.subplots(1, 2, figsize=(2, 10))
axes[0].imshow(cut_out_stamp(max_stamp)[:, :, 7])
axes[1].imshow(cut_out_stamp((120, 123))[:, :, 7])

In [ ]:
birth_halo = pd.read_csv('halos_00103.0.ascii', header=0, comment='#', delimiter=' ')
birth_halo = np.array([birth_halo.y, birth_halo.x, birth_halo.z]).T
# Convert into pixel coords
birth_halo /= coord_conversion.dl

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(coord_conversion.sm_delta[128, :, :].T)
masked_halo_plt = birth_halo[np.logical_and(birth_halo[:, 0] >= 124, birth_halo[:, 0] <= 132)]
plt.scatter(masked_halo_plt[:, 1], masked_halo_plt[:, 2], color='white', s=1)

In [ ]:
fudge_pix = 25

clamato_footprint_halo = np.array([c for c in birth_halo 
                                   if clamato_footprint_bounds[0][0] <= c[0] < clamato_footprint_bounds[0][1]
                                   and clamato_footprint_bounds[1][0] - fudge_pix <= c[1] < clamato_footprint_bounds[1][1] + fudge_pix
                                   and clamato_footprint_bounds[2][0] - fudge_pix <= c[2] < clamato_footprint_bounds[2][1] + fudge_pix])

In [ ]:
print(len(clamato_footprint_halo), len(birth_halo))

In [ ]:
def create_mock_lyapix_galcoord(lya_delta_dens, lya_mask, halo_coord_pix):
    lya_mask = lya_mask.astype(bool)
    lya_grid_pix = np.indices(lya_delta_dens.shape).transpose(1,2,3,0)
    lya_delta_masked = lya_delta_dens[lya_mask].flatten()
    lya_grid_masked = lya_grid_pix[lya_mask, :]
    assert lya_grid_masked.shape[-1] == 3 and lya_delta_masked.shape[0] == lya_grid_masked.shape[0]
    lyapix = SimpleNamespace()
    #lyapix.coord = coord_concat([coord_conversion.original_pos(*coord_conversion.index2coord(x, y, z)) for (x, y, z) in tqdm.tqdm(lya_grid_masked)])
    lyapix.coord = coord_conversion.original_pos(*coord_conversion.index2coord(lya_grid_masked[:, 0], lya_grid_masked[:, 1], lya_grid_masked[:, 2]))
    lyapix.coord = SkyCoord(ra=lyapix.coord.ra, dec=lyapix.coord.dec, distance=cosmo.comoving_distance(np.array(lyapix.coord.distance)))
    lyapix.w = np.ones_like(lya_delta_masked)
    lya_flux = np.exp(-1 * 0.226 * (lya_delta_masked + 1)**1.5)
    lyapix.delta = (lya_flux / np.mean(lya_flux)) - 1
    lyapix.npix = len(lya_delta_masked)
    
    gal_coord = coord_concat([coord_conversion.original_pos(*coord_conversion.index2coord(x, y, z)) for (x, y, z) in halo_coord_pix])
    gal_coord = SkyCoord(ra=gal_coord.ra, dec=gal_coord.dec, distance=cosmo.comoving_distance(np.array(gal_coord.distance)))
    return lyapix, gal_coord

In [ ]:
birth_lyapix, birth_coord = create_mock_lyapix_galcoord(coord_conversion.sm_delta, clamato_footprint_mask, clamato_footprint_halo)
xcorr_birth, _ = xcorr.xcorr_gal_lya(birth_coord, birth_lyapix, SigEdges, PiEdges, cosmo=constants.COSMOLOGY)

# BIRTH unconstrained sim x-corr

Compute x-corr over the whole volume, but limited in the survey's z-range. Since the sim is equally unconstrained everywhere.

In [ ]:
unconstrained_sm_delta = np.fromfile('random102_256_z2p3.dat', dtype=np.single).reshape(256, 256, 256)
unconstrained_sm_delta = scipy.ndimage.gaussian_filter(unconstrained_sm_delta, sigma=1)
unconstrained_halo = pd.read_csv('halos_random102_256_z2p3.ascii', header=0, comment='#', delimiter=' ')
unconstrained_halo = np.array([unconstrained_halo.x, unconstrained_halo.y, unconstrained_halo.z]).T
# Convert into pixel coords
unconstrained_halo /= coord_conversion.dl

In [ ]:
plt.figure(figsize=(8, 158))
plt.imshow(unconstrained_sm_delta[128, :, :].T)
masked_halo_plt = unconstrained_halo[np.logical_and(unconstrained_halo[:, 0] >= 124, unconstrained_halo[:, 0] <= 132)]
plt.scatter(masked_halo_plt[:, 1], masked_halo_plt[:, 2], color='white', s=1)

In [ ]:
if os.path.exists('xcorr_random102.npy'):
    xcorr_unconstrained = np.load('xcorr_random102.npy')
else:
    unconstrained_lyapix, unconstrained_coord = create_mock_lyapix_galcoord(unconstrained_sm_delta, np.ones_like(unconstrained_sm_delta), unconstrained_halo)
    xcorr_unconstrained, _ = xcorr.xcorr_gal_lya(unconstrained_coord, unconstrained_lyapix, SigEdges, PiEdges, cosmo=constants.COSMOLOGY)
    np.save('xcorr_random102.npy', xcorr_unconstrained.value)

In [ ]:
X, Y = np.meshgrid(SigEdges, PiEdges)

SigMax = 35.
PiMin = -40.
PiMax = 40.

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8,14))

print(len(X), len(Y), np.shape(xcorr_birth.T))

ax1.pcolormesh(X, Y, xcorr_birth.T,cmap='magma_r',vmin=-0.3, vmax=0.1)
ax1.set_aspect('equal')
ax1.set_xlim(np.min(X), SigMax)
ax1.set_ylim(PiMin, PiMax)

ax2.pcolormesh(X, Y, xcorr_unconstrained.T,cmap='magma_r',vmin=-0.3, vmax=0.1)
ax2.set_aspect('equal')
ax2.set_xlim(np.min(X), SigMax)
ax2.set_ylim(PiMin, PiMax)